# Analysis of multiple isotopic contrasts

In the use of neutron reflectometry, it is common to use multiple isotopic contrasts of experimental data to analyse a system in a constrained fashion. 
That is to say, that we have data from multiple different species (where isotopic substitution has been used to produce data with different scattering length density) that shar ethe same structure. 
In this tutorial we will look at how `EasyReflectometry` can be used to fit multiple contrasts of data from a [surfactant monolayer](./monolayer.html) system, if you haven't looked at the tutorial for a single contrast of surfactant monolayer data it is suggested that you as this tutorial will build on it.
First, as always, we import the relevant packages and print the version numbers.

In [ ]:
import EasyReflectometry
import refnx
from EasyReflectometry.data import load
from EasyReflectometry.plot import plot
from EasyReflectometry.sample.material import Material
from EasyReflectometry.sample.item import SurfactantLayer
from EasyReflectometry.sample import Layer, Structure
from EasyReflectometry.experiment.model import Model
from EasyReflectometry.interface import InterfaceFactory as Interface
from EasyReflectometry.fitting import Fitter
print(f'EasyReflectometry: {EasyReflectometry.__version__}')
print(f'refnx: {refnx.__version__}')

## Reading in the experimental data

We load in three different isotopic contrast that are stored in a single `.ort` file. 
This `.ort` file uses the [mutliple data set syntax](https://github.com/reflectivity/file_format/blob/master/specification.md#multiple-data-sets) to indicate that different measurements are present in a single file. 

In [ ]:
data = load('_static/multiple.ort')

We can plot the data and return the dimensions of the data to see the contrasts that are present. 

In [ ]:
data.dims

In [ ]:
plot(data)

## Building our models

It can be seen above that there are three isotopic contrasts present in the `'multiple.ort'` file, namely:
- `'d13DSPC-D2O'`
- `'d70DSPC-D2O'`
- `'d83DSPC-ACMW'`

where, `'d13'` indicates that it is the head layer that is deuterated, `'d70'` indicates that the tail is deuterated and `'d83'` indicates the whole molecule is deuterated. 
We describe these different deuterations as chemical formulae below. 

In [ ]:
dspc = {'d-head': 'C10D18NO8P', 'd-tail': 'C34D70',
        'h-head': 'C10H18NO8P', 'h-tail': 'C34H70'}

The solvent contrast is indicated as either D2O or ACMW (air-contrast matched water). 

In [ ]:
d2o = Material.from_pars(6.36, 0, 'D2O')
acmw = Material.from_pars(0, 0, 'ACMW')

All of the contrasts are at the air/water interface, so we also need an `air` material.

In [ ]:
air = Material.from_pars(0, 0, 'Air')

Then we can create all of the traditional layers that we will need. 

In [ ]:
d2o_layer = Layer.from_pars(d2o, 0, 3, 'D2O Subphase')
acmw_layer = Layer.from_pars(acmw, 0, 3, 'D2O Subphase')
air_layer = Layer.from_pars(air, 0, 0, 'Air Superphase')

The different isotopic contrasts of `SurfactantLayer` objects can also be created (all with the same structural parameters).

In [ ]:
head_thickness = 12
tail_thickness = 20
head_solvation = 0.5
tail_solvation = 0.0
area_per_molecule = 45
roughness = 3

In [ ]:
d13d2o = SurfactantLayer.from_pars(dspc['h-tail'], tail_thickness, d2o, tail_solvation, 
                                   area_per_molecule, roughness,
                                   dspc['d-head'], head_thickness, d2o, head_solvation, 
                                   area_per_molecule, roughness)
d13d2o.constrain_apm = True
d13d2o.conformal_roughness = True
d13d2o.constrain_solvent_roughness(d2o_layer)
d70d2o = SurfactantLayer.from_pars(dspc['d-tail'], tail_thickness, d2o, tail_solvation, 
                                   area_per_molecule, roughness,
                                   dspc['h-head'], head_thickness, d2o, head_solvation, 
                                   area_per_molecule, roughness)
d70d2o.constrain_apm = True
d70d2o.conformal_roughness = True
d70d2o.constrain_solvent_roughness(d2o_layer)
d83acmw = SurfactantLayer.from_pars(dspc['d-tail'], tail_thickness, acmw, tail_solvation, 
                                    area_per_molecule, roughness,
                                    dspc['d-head'], head_thickness, acmw, head_solvation, 
                                    area_per_molecule, roughness)
d83acmw.constrain_apm = True
d83acmw.conformal_roughness = True
d83acmw.constrain_solvent_roughness(acmw_layer)

## Introducing constraints

Then to ensure that the structure (thicknesss, area per molecule, etc.) is kept the same between the different contrasts we constain these (`layer2` is the head layer and `layer1`, which the neutron are incident on first are the tail layer). 
The `constrain_multiple_contrast` method allows this, not that is it important that a chain of constraints is produced, one constraining the next. 

In [ ]:
# These four lines should be removed in future
d70d2o.layers[1].area_per_molecule.enabled = True
d70d2o.layers[0].area_per_molecule.enabled = True
d83acmw.layers[1].area_per_molecule.enabled = True
d83acmw.layers[0].area_per_molecule.enabled = True

d70d2o.constain_multiple_contrast(d13d2o)
d83acmw.constain_multiple_contrast(d70d2o)

We can check this constraint as worked as follows.

In [ ]:
d13d2o.layers[1].thickness.raw_value, d70d2o.layers[1].thickness.raw_value, d83acmw.layers[1].thickness.raw_value

In [ ]:
d13d2o.layers[1].thickness.value = 10

In [ ]:
d13d2o.layers[1].thickness.raw_value, d70d2o.layers[1].thickness.raw_value, d83acmw.layers[1].thickness.raw_value

Even through only as single value (that for the d13-DSPC head thickness) was changed, all three values changed. 

Having constructed each of the surfactant layer object and implemented the constraints, we can now build structures and models. 

In [ ]:
d13d2o_structure = Structure.from_pars(air_layer, d13d2o, d2o_layer)
d70d2o_structure = Structure.from_pars(air_layer, d70d2o, d2o_layer)
d83acmw_structure = Structure.from_pars(air_layer, d83acmw, acmw_layer)
d13d2o_model = Model.from_pars(d13d2o_structure, 0.1, data['R_d13DSPC-D2O'].data.values.min(), 5)
d70d2o_model = Model.from_pars(d70d2o_structure, 0.1, data['R_d70DSPC-D2O'].data.values.min(), 5)
d83acmw_model = Model.from_pars(d83acmw_structure, 0.1, data['R_d83DSPC-ACMW'].data.values.min(), 5)

## Setting varying parameters

For this analysis, we want the scale and background for each model to vary and then some of the structural parameters. 
Since the structural parameters are constrained, we only need to define the bounds once. 
In the case on the `d13d2o` object, as this is the basis for the constraints. 

In [ ]:
d13d2o_model.scale.bounds = (0.05, 1.5)
d13d2o_model.background.bounds = (4e-8, 1e-5)
d70d2o_model.scale.bounds = (0.05, 1.5)
d70d2o_model.background.bounds = (4e-8, 1e-5)
d83acmw_model.scale.bounds = (0.05, 1.5)
d83acmw_model.background.bounds = (4e-8, 1e-5)

d13d2o.layers[0].area_per_molecule.bounds = (40, 50)
d13d2o.layers[1].solvation.bounds = (0.2, 0.6)
d13d2o.layers[0].thickness.bounds = (18, 24)
d13d2o.layers[1].thickness.bounds = (8, 12)

## Creating interfaces and performing the fitting

The model has been created and the parameters defined, so the fitting process can begin. 
First, we should create a unique interface for each of the models. 

In [ ]:
d13d2o_model.interface = Interface()
d70d2o_model.interface = Interface()
d83acmw_model.interface = Interface()

The models and the interface fit functions are then passed to the `Fitter` object, which is capable of performed the multiple dataset fitting process.

In [ ]:
f = Fitter(d13d2o_model, d70d2o_model, d83acmw_model)

In [ ]:
analysed = f.fit(data)

Once the fitting is complete, we can probe a given value to check that the constraints have been respected or plot the results.

In [ ]:
d13d2o.layers[1].area_per_molecule.raw_value, d70d2o.layers[1].area_per_molecule.raw_value, d83acmw.layers[1].area_per_molecule.raw_value

In [ ]:
plot(analysed)